<a href="https://colab.research.google.com/github/arahrooh31/UCLA_BE223C/blob/Keane_temp/VGG_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#VGG_CLASSIFY


*   This function will take in an input file and ROI and process a 3 class classification 
*   The final output will be a Cancer, No Cancer decision for the ROI given

USAGE:




In [7]:
import os
import sys  #to set local import folder
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import gc  #debug memory leaks in matplotlib
import csv #read in description files

from google.colab.patches import cv2_imshow


import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
#show model design parameters with torchsummary
import torchsummary
from torchsummary import summary
from torch import FloatTensor
from torch import tensor
from torch.utils.data import DataLoader

#
# Read Data from google drive
#
from google.colab import drive #for loading gdrive data
from google.colab import files
################################################################################
#
# Load data from google drive
#

drive.mount('/content/gdrive')

#
# SET LOCAL IMPORT DIRECTORY FOR .PY FILES
#
sys.path.append('/content/gdrive/My Drive/DBT_WORKSPACE/DBT_PY_FILES')
!ls '/content/gdrive/My Drive/DBT_WORKSPACE/DBT_PY_FILES'
from CustomImageDataset import CustomImageDataset
from VGG16 import VGG16

# install dependencies not included by Colab
# use pip3 to ensure compatibility w/ Google Deep Learning Images 
!pip3 install -q pydicom 
!pip3 install -q tqdm 
!pip3 install -q imgaug
!pip3 install -q pickle5

import pydicom #to read dicom files
from pydicom import dcmread
import pickle5 as pickle; #generic storage of image array




### Enable GPU, if present
train_on_gpu = torch.cuda.is_available()
if (train_on_gpu):
    !nvidia-smi -L
    !nvidia-smi 
    dev=torch.device("cuda")
else:
    print('GPU NOT FOUND!!! USING CPU INSTEAD!!!!!')




data_dir = '/content/gdrive/My Drive/DBT_DATA/IMG_ARRAYS'
patch_dir = '/content/gdrive/My Drive/DBT_WORKSPACE/TRAINING_PATCHES' 
model_dir = '/content/gdrive/My Drive/BE223C_SPRING_2021/MODEL_SAVE'
tensorboard_dir = '/content/gdrive/My Drive/BE223C_SPRING_2021/TENSORBOARD_SUMMARIES'






#from get_dirs import get_dirs



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
CustomImageDataset.py  __pycache__  VGG16.py
GPU NOT FOUND!!! USING CPU INSTEAD!!!!!


In [ ]:
#
# Read inputs
#

In [ ]:
################################################################################
# GET FULL LIST OF FILES IN IMAGE ARRAY DIRECTORY
################################################################################
use_patch_files = 1
if (use_patch_files == 0):
    raw_files = os.listdir(data_dir)
    print('found #files: ',len(raw_files))
else: #patches broken up into directories
    category_folders = os.listdir(patch_dir)
    #raw_files = os.listdir(data_dir)
    #print('found #files: ',len(raw_files))    

if (0):
    #create fake patches for now
    patch_dict = {}
    for counter,filename in enumerate(raw_files):
        #load full array
        full_filename = os.path.join(data_dir,filename)
        img_data = pickle.load( open( full_filename, "rb" ) )
        patch_data = img_data[0:3,:,:]
        patch_dict[counter]= patch_data
        print(full_filename,np.shape(patch_data))

        if (counter > 0):
            break
        
### SKIP ACTIONABLE FOLDER
### Remove actionable folder item from list, since we're no longer using that data
category_folders.remove('ACTIONABLE')

In [8]:
for ii in category_folders:
    print(ii)
    flist = os.listdir(os.path.join(patch_dir,ii))

temp = flist[0].split(sep='_')
print(temp[3])

NameError: ignored

In [ ]:
#generate full file list for use in indexing the dataloader
#this replaces the older loader, which was only inputting Normal
full_file_list = [] #store the full filename of every file
full_category_name = []
for category_folder in category_folders:
            print('------------- ',category_folder)
            file_list = os.listdir(os.path.join(patch_dir,category_folder))
            cat_count = 0
            for file_name in file_list:
                cat_count = cat_count + 1
                full_category_name.append(category_folder)
                full_file_list.append(file_name)
                print(os.path.join(category_folder,file_name))
full_file_count = len(full_file_list)
print(len(full_file_list))
full_file_list[0]

In [ ]:
#Show summary of model setup and move model to the GPU
 #train_on_gpu = torch.cuda.is_available()

bsize = 50

model_vgg16 = VGG16() #.to(device)
model_vgg16 = model_vgg16.float()
print(model_vgg16)


if (train_on_gpu == 1):
    #dev=torch.device("cuda") 
    model_vgg16.to(dev)
    summary(model_vgg16,(3,244,244), batch_size = bsize, device='cuda')
else:
    summary(model_vgg16,(3,244,244), batch_size = bsize)

In [ ]:
model_name = 'vgg16_best_accuracy_93_EPOCH_96_0.04582521319389343'
index_storage_file = os.path.join(tensorboard_dir,'data_index_052521_gpu.pickle')
index_file = os.path.join(tensorboard_dir,'data_index_052521_gpu.pickle') #052021_gpu.pickle') #data_index_last.pickle')

In [ ]:
##########################################################
# Test against the test dataset to get accuracy results
##########################################################
run_test= 1
if (run_test == 1):
    #Mixing GPU and CPU model saves doesn't seem to map well yet
    final_file = os.path.join(model_dir,model_name) #'vgg16_best_accuracy_81_EPOCH_79') #95_gpu_051821')#'vgg16_best_accuracy_97_gpu_final')
    checkpoint = torch.load(final_file, map_location=torch.device('cpu'))



    
    use_index = 1
    if (use_index == 1):
        #[val_index, test_index, file_list]
        validation_saved_index,test_saved_index,saved_flist=pickle.load( open( index_file, "rb" )) 
        test_files=[]
        for ii in test_saved_index:
            test_files.append(full_file_list[ii])
        print('!!!! Using Test Indexed Files ONLY !!!!')
    else: #use full files      
        bc_files=[]
        for ii in full_file_list:
            if (('Benign' in ii) or ('Cancer' in ii)):
                bc_files.append(ii)
                #print(ii)
    
    #new_file_list=[]
    #for ii in b:
    #    new_file_list.append(full_file_list[ii])


    model_vgg16.load_state_dict(checkpoint)
    model_vgg16.eval()


    #load up with the pre-sized patch images
    all_data = CustomImageDataset(img_dir=patch_dir,
                                    category = full_category_name, 
                                    file_count=len(test_files), #full_file_count,
                                    file_list = test_files, #flist, #new_file_list, #full_file_list, 
                                    transform=None, 
                                    target_transform=None)

    dataloader_all = DataLoader(all_data, batch_size=bsize,shuffle=True, num_workers=2)#, 

    total_accuracy = []
    total_precision = []
    total_average_precision=[] 
    total_final_accuracy = []
    total_final_precision = []
    total_final_recall_sens = []
    total_f1_score =[]
    for epoch in range(0,1):
        with torch.no_grad():
            for i, data in enumerate(dataloader_all, 0):
                print(i)
                # get the inputs; data is a list of [inputs, labels]
                #inputs, labels = data
                inputs = data['image'].type(FloatTensor)
                labels = data['label'] #.type(FloatTensor)

                if (train_on_gpu):
                    inputs, labels = inputs.to(dev), labels.to(dev)

                # forward + backward + optimize
                outputs = model_vgg16(inputs) #.permute(0, 1, 2, 3))

                outputs=torch.flatten(outputs, start_dim=1)
                loss = criterion(outputs, labels.long())


                #print(outputs)
                y_pred_softmax = torch.log_softmax(outputs, dim = 1)
                _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
                #print(y_pred_tags)
                #print(labels)
                correct_pred = (y_pred_tags == labels).float()
                accuracy = correct_pred.sum() / len(correct_pred)
                #accuracy = torch.round(accuracy)


                #Get advanced metrics
                average_precision, final_accuracy, final_precision, final_recall_sens, f1_score = calculate_metrics(labels,y_pred_tags)

                total_average_precision.append(average_precision) 
                total_final_accuracy.append(final_accuracy)
                total_final_precision.append(final_precision)
                total_final_recall_sens.append(final_recall_sens)
                total_f1_score.append(f1_score)

                total_accuracy.append(accuracy)
                
                if (i%100 == 0):
                    print('@ interim accuracy = ',i,  sum(total_accuracy)/len(total_accuracy))                
                #print('-----#correct, training accuracy = ',correct_pred,accuracy)
    print('Finished testing all data')
    print('total accuracy = ', sum(total_accuracy)/len(total_accuracy))
    test_p_file = os.path.join(tensorboard_dir,'test_metrics_debug.pickle')
    pickle.dump([total_average_precision,total_final_accuracy, 
                 total_final_precision, total_final_recall_sens, 
                 total_f1_score, total_accuracy],
                 open( test_p_file, "wb" ),protocol=5 )

In [ ]:
def vgg_classify():
    return 0